In [16]:
from nyt_article_search import JSONParse
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as f
from pyspark.sql.types import *
from itertools import chain
from spark_job_functions import *
import re
import pandas as pd
import json
import os

### Load JSONs and transform in to lists of tuples

In [17]:
# Lists that the data from each json response will be added to
article_ids = []
fact_data = []
author_data = []
subject_data = []
people_data = []
org_data = []
loc_data = []
# Big text will be a dictionary where we'll add headline, lead paragraph,
# abstract and web url
big_text = {}

In [18]:
data_folder = 'DATA'
state = 'TEXAS'
# states = [f for f in os.listdir(data_folder) if f != '_Archive']
state_folder = os.path.join(data_folder, state)
json_files = os.listdir(state_folder)
for file in json_files:
    filepath = os.path.join(state_folder, file)
    json_file = open(filepath, 'r')
    data = json.load(json_file)
    responses = data['response']['docs']
    # Parse each json response in the json file
    for respsonse in responses:
        j = JSONParse(respsonse)
        article_ids.append(j.article_id)
        fact_data.append(j.get_article_facts())
        extend_list(author_data, j.get_article_authors())
        extend_list(subject_data, j.search_article_keywords('subject'))
        extend_list(people_data, j.search_article_keywords('persons'))
        extend_list(org_data, j.search_article_keywords('organizations'))
        extend_list(loc_data, j.search_article_keywords('glocations'))
        id_text_dict = {
            j.article_id:{
                            'headline':j.get_text('headline')[1],
                            'abstract':j.get_text('abstract')[1],
                            'lead_paragraph':j.get_text('lead_paragraph')[1],
                            'web_url':j.get_text('web_url')[1]
                        }
            }
        big_text.update(id_text_dict)

In [19]:
spark = SparkSession.\
        builder.\
        appName('NYT_JSON_ETL').\
        master('local[1]').\
        getOrCreate()

In [20]:
# #For each result get the column headers for the resulting
# #dataframe
# def get_table_headers(table):

#     table_col_dict = {
#         'facts':['article_id', 'publication_date',
#                             'word_count', 'total_keywords',
#                             'total_authors', 'words_in_headline',
#                             'in_print', 'print_page', 'print_section',
#                             'news_desk', 'section_name', 'article_type'],
#         'authors':['article_id', 'rank', 'role',
#                             'firstname', 'middlename', 'lastname', 'qualifier'],
#         'subjects':['article_id', 'rank', 'name', 'value', 'major'],
#         'text':['article_id', 'text']
#     }

#     headers = table_col_dict.get(table)
#     return(headers)

In [21]:
#Create spark dfs
facts = spark.createDataFrame(fact_data, schema=get_table_headers('facts'))
authors = spark.createDataFrame(author_data, schema=get_table_headers('authors'))
subjects = spark.createDataFrame(subject_data, schema=get_table_headers('subjects'))
people = spark.createDataFrame(people_data, schema=get_table_headers('subjects'))
organizations = spark.createDataFrame(org_data, schema=get_table_headers('subjects'))
locations = spark.createDataFrame(loc_data, schema=get_table_headers('subjects'))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\1saml\\AppData\\Local\\Temp\\spark-6d220d61-7512-49a5-b1a4-f20c1486129f\\pyspark-6218f323-9b96-4bee-b90e-33ec6ad7fabc\\tmpvvatmjsg'

### Step 1: Create Primary Keys in tables

In [ ]:
# First step is to create an interger primary key for the article_ids
ids = [(e + 1000, i) for e, i in enumerate(article_ids)]
# Create spark df out of id list
# Integer Primary Key is fact_id
id_schema = StructType([
    StructField('fact_id', IntegerType(), False),
    StructField('article_id', StringType(), False)
    ])

### THIS SHOULD NOT CHANGE ANY FURTHER
id_df = spark.createDataFrame(ids, schema=id_schema)



#Merge the id_df dataframe into the existing frames
# To put fact_id in all the other tables
facts = id_df.join(facts, ['article_id'], how = 'inner').drop('article_id')
authors = id_df.join(authors, ['article_id'], how = 'inner').drop('article_id')
subjects = id_df.join(subjects, ['article_id'], how = 'inner').drop('article_id')
people = id_df.join(people, ['article_id'], how = 'inner').drop('article_id')
organizations = id_df.join(organizations, ['article_id'], how = 'inner').drop('article_id')
locations = id_df.join(locations, ['article_id'], how = 'inner').drop('article_id')

In [ ]:
# Union the subjects, organizations, and locations dfs togehter
places_and_things = subjects.union(organizations).union(locations).orderBy(['fact_id', 'rank'])

#

In [ ]:

places_and_things = create_primary_key(places_and_things, 'table_id', 'fact_id', 'rank')
authors = create_primary_key(authors, 'table_id', 'fact_id', 'rank')
people = create_primary_key(people, 'table_id', 'fact_id', 'rank')
places_and_things.show(2)
authors.show(2)
people.show(2)


+-------+----+----------+------+-----+--------+
|fact_id|rank|      name| value|major|table_id|
+-------+----+----------+------+-----+--------+
|   1000|   1|glocations| Texas|    N|1000.001|
|   1000|   3|   subject|Ethics|    N|1000.002|
+-------+----+----------+------+-----+--------+
only showing top 2 rows

+-------+----+--------+---------+----------+--------+---------+--------+
|fact_id|rank|    role|firstname|middlename|lastname|qualifier|table_id|
+-------+----+--------+---------+----------+--------+---------+--------+
|   1000|   1|reported|    Steve|    Barnes|     NYT|     NULL|1000.001|
|   1003|   1|reported|  Michael|      NULL|    NULL|     NULL|1003.001|
+-------+----+--------+---------+----------+--------+---------+--------+
only showing top 2 rows



+-------+----+-------+----------------+-----+--------+
|fact_id|rank|   name|           value|major|table_id|
+-------+----+-------+----------------+-----+--------+
|   1000|   2|persons|ROACH, RICHARD J|    N|1000.001|
|   1001|   5|persons| Trump, Donald J|    N|1001.001|
+-------+----+-------+----------------+-----+--------+
only showing top 2 rows



### Step 2: Text standardization

In [ ]:
# Read in dimensional news desk, section name, and article type files
dim_folder = os.path.join('DATA', 'DIM_TABLES')
dim_news_desk = os.path.join(dim_folder, 'news_desks.csv')
dim_types = os.path.join(dim_folder, 'article_types.csv')
dim_sections = os.path.join(dim_folder, 'section_names.csv')

news_desk_df = spark.read.option('header', True).csv(dim_news_desk)
material_df = spark.read.option('header', True).csv(dim_types)
section_df = spark.read.option('header', True).csv(dim_sections)

In [ ]:
#standardize any text in the article_type, news_desk, and section_name
#columns that aren't in the dimensinal tables
#EX: National Desk vs National - should be National
facts = standardize_text(facts, 'article_type', material_df)
facts = standardize_text(facts, 'news_desk', news_desk_df)
facts = standardize_text(facts, 'section_name', section_df)
facts.show(5)


+-------+----------------+----------+--------------+-------------+-----------------+--------+----------+-------------+---------+------------+-------------+
|fact_id|publication_date|word_count|total_keywords|total_authors|words_in_headline|in_print|print_page|print_section|news_desk|section_name| article_type|
+-------+----------------+----------+--------------+-------------+-----------------+--------+----------+-------------+---------+------------+-------------+
|   1083|      2015-11-13|       564|             5|            1|                8|    true|         4|           TR|   Travel|      Travel|         News|
|   1062|      2010-04-08|       853|             6|            1|                7|    true|        21|            A| National|     Science|         News|
|   1005|      2000-11-12|      1003|             6|            1|               11|    true|        22|            1| National|        U.S.|         News|
|   1039|      2005-09-29|      1133|            18|            

In [ ]:
#clean the print_page column in facts df
#Some print pages aren't numeric: 25A - A is already in print_section column
#Create a list of unique page numbers 
page_rdd = facts.select(['print_page']).distinct().rdd.flatMap(lambda x: x).collect()
#include pages that aren't None
pages = [p for p in set(page_rdd) if p != None]

pages_cleaned = {}
for page in pages:
    if page != None:
        try:
            p = float(page)
            p_dict = {page:p}
            pages_cleaned.update(p_dict)
        except:
            p = page[:-1]
            p = float(p)
            p_dict = {page:p}
            pages_cleaned.update(p_dict)
    else:
        pass
#create mapping
mapping = f.create_map([f.lit(x) for x in chain(*pages_cleaned.items())])

#update the print_page column with mapping
facts = facts.withColumn('print_page', mapping[facts['print_page']])
facts.show(5)


+-------+----------------+----------+--------------+-------------+-----------------+--------+----------+-------------+---------+------------+-------------+
|fact_id|publication_date|word_count|total_keywords|total_authors|words_in_headline|in_print|print_page|print_section|news_desk|section_name| article_type|
+-------+----------------+----------+--------------+-------------+-----------------+--------+----------+-------------+---------+------------+-------------+
|   1083|      2015-11-13|       564|             5|            1|                8|    true|       4.0|           TR|   Travel|      Travel|         News|
|   1062|      2010-04-08|       853|             6|            1|                7|    true|      21.0|            A| National|     Science|         News|
|   1005|      2000-11-12|      1003|             6|            1|               11|    true|      22.0|            1| National|        U.S.|         News|
|   1039|      2005-09-29|      1133|            18|            

In [ ]:
# IN the people df create a standardized first name, middle name, and last name columns

# Get the last name (left of comma) and upper case it
people = people.withColumn('last_name', f.upper(f.substring_index('value', ',', 1)))
# Get the first and middle name (right of comma) and upper case it
people = people.withColumn('first_middle', f.upper(f.substring_index('value', ',', -1)))
# Replace special characters in first and middle names with empty string,
# But keep inner white space to separate middle name from first
people = people.withColumn('first_middle', f.trim(f.regexp_replace('first_middle', '[^a-zA-Z ]', '')))
# Identify name qualifiers - SR, JR, II, III, IV, V
# EX - Dale Earnhardt vs Dale Earnhardt Jr
qualifier_list = ['JR', 'SR', 'II', 'III', 'IV', 'V']
qualifiers = people.where(f.substring_index('first_middle', ' ', -1).isin(qualifier_list)).select(['table_id', 'first_middle'])

#Map replacement string for names with ending qualifiers
name_replace = qualifiers.select('first_middle')
name_list = name_replace.select(['first_middle']).distinct().rdd.flatMap(lambda x: x).collect()
name_dict = {name:name[0:name.rindex(' ')] for name in name_list}
#create mapping
name_map = f.create_map([f.lit(x) for x in chain(*name_dict.items())])
people = people.withColumn('first_middle', name_map[people['first_middle']])

#Get middle name / initial
people = people.withColumn('middle', 
    f.when(f.size(f.split(people['first_middle'], ' ', -1)) == 1, None).otherwise(f.substring_index(people['first_middle'], ' ', -1)))

#Merge qualifiers back in as a separate column
qualifiers = qualifiers.withColumn('qualifier', f.substring_index('first_middle', ' ', -1)).select('table_id', 'qualifier')
people = people.join(qualifiers, ['table_id'], how = 'left')

people.show()

+--------+-------+----+-------+--------------------+-----+---------+------------+------+---------+
|table_id|fact_id|rank|   name|               value|major|last_name|first_middle|middle|qualifier|
+--------+-------+----+-------+--------------------+-----+---------+------------+------+---------+
|1006.002|   1006|   8|persons|      Bush, George W|    N|     BUSH|        NULL|  NULL|     NULL|
|1001.002|   1001|   7|persons|Kelley, Devin P (...|    N|   KELLEY|        NULL|  NULL|     NULL|
|1005.002|   1005|   4|persons|            Gore, Al|    N|     GORE|        NULL|  NULL|     NULL|
|1013.001|   1013|   1|persons|      Keller, Sharon|    N|   KELLER|        NULL|  NULL|     NULL|
|1014.001|   1014|   4|persons|        Davis, Wendy|    N|    DAVIS|        NULL|  NULL|     NULL|
|1002.001|   1002|   3|persons| Gonzales, Alberto R|    N| GONZALES|        NULL|  NULL|     NULL|
|1007.002|   1007|   5|persons| Gamble, Maya Guerra|    N|   GAMBLE|        NULL|  NULL|     NULL|
|1012.001|

In [ ]:
people.show()

+--------+-------+----+-------+--------------------+-----+---------+------------+------+---------+
|table_id|fact_id|rank|   name|               value|major|last_name|first_middle|middle|qualifier|
+--------+-------+----+-------+--------------------+-----+---------+------------+------+---------+
|1006.002|   1006|   8|persons|      Bush, George W|    N|     BUSH|        NULL|  NULL|     NULL|
|1001.002|   1001|   7|persons|Kelley, Devin P (...|    N|   KELLEY|        NULL|  NULL|     NULL|
|1005.002|   1005|   4|persons|            Gore, Al|    N|     GORE|        NULL|  NULL|     NULL|
|1013.001|   1013|   1|persons|      Keller, Sharon|    N|   KELLER|        NULL|  NULL|     NULL|
|1014.001|   1014|   4|persons|        Davis, Wendy|    N|    DAVIS|        NULL|  NULL|     NULL|
|1002.001|   1002|   3|persons| Gonzales, Alberto R|    N| GONZALES|        NULL|  NULL|     NULL|
|1007.002|   1007|   5|persons| Gamble, Maya Guerra|    N|   GAMBLE|        NULL|  NULL|     NULL|
|1012.001|

In [ ]:
[name[0:name.rindex(' ')] for name in name_list]

['JOSEPH R', 'WILLIAM R']